In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import json

In [2]:
from konlpy.tag import Okt
okt = Okt()

In [3]:
openFileName = '전자담배 중독_naver_news_1000.json'
    
f = open(openFileName, 'r', encoding='utf-8').read()

jsonData = json.loads(f)
data = ''

for item in jsonData:
    if 'description' in item.keys():
        data = data + re.sub(r'[^\w]', ' ', item['description']) + ' '
        
# f.close()

In [4]:
def tokenizer(raw, pos=["Noun"], stopword=[]): # 다른 품사 추가 원할 시 pos=["Noun", "Josa", ...]
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

documents1 = tokenizer(data)

In [5]:
import gensim

documents2 = [documents1] # gensim.corpora.Dictionary() 는 매개변수로 리스트 타입을 받기 때문에
dictionary = gensim.corpora.Dictionary(documents2)
print(dictionary.token2id)
print('dictionary size : %d' % len(dictionary)) # dictionary size : 37987

{'가게': 0, '가격': 1, '가공': 2, '가까이': 3, '가늠': 4, '가능성': 5, '가도': 6, '가동': 7, '가량': 8, '가명': 9, '가미': 10, '가스': 11, '가야': 12, '가연성': 13, '가열': 14, '가운데': 15, '가이드라인': 16, '가장': 17, '가정의학': 18, '가족': 19, '가주': 20, '가지': 21, '가파르': 22, '가향': 23, '각각': 24, '각국': 25, '각성': 26, '각심': 27, '각종': 28, '간다': 29, '간접': 30, '간접흡연': 31, '간판': 32, '감동': 33, '감사원': 34, '감소': 35, '감안': 36, '감축': 37, '강도': 38, '강력': 39, '강사': 40, '강서구': 41, '강아지': 42, '강요': 43, '강의': 44, '강조': 45, '강화': 46, '개국': 47, '개꼴': 48, '개발': 49, '개비': 50, '개선': 51, '개월': 52, '개인': 53, '개입': 54, '개정': 55, '개정안': 56, '개탄': 57, '개편': 58, '개피': 59, '객관': 60, '갤럽': 61, '거나': 62, '거리': 63, '거세': 64, '거울': 65, '거의': 66, '거주': 67, '걱정': 68, '건강': 69, '건국대': 70, '건의': 71, '검사': 72, '검색': 73, '검열': 74, '검은색': 75, '검토': 76, '게다가': 77, '게임': 78, '게재': 79, '게티': 80, '겨냥': 81, '격납고': 82, '격전': 83, '견해': 84, '결과': 85, '결론': 86, '결심': 87, '결정': 88, '결코': 89, '결합': 90, '결혼': 91, '결혼식': 92, '경고': 93, '경련': 94, '경상': 95, '경우': 96, '경제': 97, '경제성': 9

In [6]:
corpus_path = '전자담배 중독_naver_news_1000.json'

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

documents = Documents(corpus_path)

In [7]:
class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length

corpus = Corpus(corpus_path, dictionary)
for i, doc in enumerate(corpus):
    #if i >= 5: break
    print(doc)

[]
[]
[(202, 1), (278, 1), (708, 1), (738, 1), (1112, 1), (1129, 1), (1369, 1), (1371, 1), (1453, 1)]
[]
[]
[]
[]
[]
[]
[(210, 1), (309, 1), (448, 1), (1079, 1), (1682, 1)]
[]
[]
[]
[(309, 1)]
[]
[]
[(202, 1), (278, 1), (708, 1), (738, 1), (1112, 1), (1129, 1), (1369, 1), (1371, 1), (1453, 1)]
[]
[]
[]
[(1831, 1)]
[]
[]
[(69, 1), (1217, 1), (1232, 1)]
[]
[]
[]
[(285, 1)]
[]
[]
[(1547, 1), (1575, 1)]
[]
[]
[]
[]
[]
[]
[(182, 1), (249, 1), (834, 1), (1235, 1), (1305, 1), (1589, 1)]
[]
[]
[]
[(182, 1), (764, 1)]
[]
[]
[(78, 1), (175, 1), (1067, 1)]
[]
[]
[]
[(78, 1)]
[]
[]
[(334, 1), (584, 1), (715, 1), (910, 1), (946, 1), (1042, 1), (1232, 1)]
[]
[]
[]
[(1275, 1)]
[]
[]
[(397, 1), (555, 2), (714, 1), (1232, 1)]
[]
[]
[]
[(555, 1)]
[]
[]
[(820, 1), (901, 1), (1235, 1), (1622, 1)]
[]
[]
[]
[]
[]
[]
[(334, 1), (910, 1), (946, 1), (1232, 1)]
[]
[]
[]
[(1275, 1)]
[]
[]
[(20, 2), (1232, 1)]
[]
[]
[]
[]
[]
[]
[(910, 1)]
[]
[]
[]
[]
[]
[]
[(111, 1), (289, 2)]
[]
[]
[]
[(289, 1)]
[]
[]
[(289, 1),

[]
[]
[]
[]
[]
[]
[(334, 2), (1125, 1), (1232, 1)]
[]
[]
[]
[(380, 1), (1711, 1)]
[]
[]
[(334, 2), (1232, 1)]
[]
[]
[]
[]
[]
[]
[(334, 2), (1232, 1)]
[]
[]
[]
[]
[]
[]
[(334, 2), (1232, 1)]
[]
[]
[]
[]
[]
[]
[(338, 1), (553, 1), (1121, 1), (1771, 1), (1826, 1)]
[]
[]
[]
[]
[]
[]
[(334, 1), (353, 1), (1322, 1)]
[]
[]
[]
[(225, 1), (1639, 1)]
[]
[]
[(334, 1), (1400, 1), (1853, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(334, 1), (1232, 1), (1339, 1)]
[]
[]
[]
[]
[]
[]
[(212, 1), (334, 2), (377, 1), (1400, 1), (1790, 2)]
[]
[]
[]
[(295, 1), (1600, 1)]
[]
[]
[(334, 2), (730, 1), (1112, 1)]
[]
[]
[]
[]
[]
[]
[(201, 1), (334, 1), (1856, 1)]
[]
[]
[]
[(746, 1)]
[]
[]
[(23, 1), (202, 1), (334, 1), (397, 1), (821, 1), (919, 1), (1686, 1)]
[]
[]
[]
[(23, 1), (375, 1), (1521, 1), (1659, 1)]
[]
[]
[(555, 2), (1653, 1)]
[]
[]
[]
[(555, 1), (1521, 1)]
[]
[]
[(821, 1), (1440, 1)]
[]
[]
[]
[(225, 1), (386, 1)]
[]
[]
[(206, 1), (225, 1)]
[]
[]
[]
[(225, 1), (1158, 1)]
[]
[]
[(202, 1), (821, 1), (1639,

[]
[(913, 1)]
[]
[]
[(158, 1), (1222, 1), (1618, 1)]
[]
[]
[]
[]
[]
[]
[(1222, 1), (1618, 1)]
[]
[]
[]
[]
[]
[]
[(328, 1), (792, 1), (890, 2), (1232, 2)]
[]
[]
[]
[]
[]
[]
[(334, 2)]
[]
[]
[]
[(645, 1)]
[]
[]
[(182, 1), (1232, 1), (1298, 1), (1433, 1), (1639, 1), (1659, 1)]
[]
[]
[]
[]
[]
[]
[(249, 1), (850, 1), (1486, 1)]
[]
[]
[]
[(998, 1)]
[]
[]
[(271, 1), (334, 1), (491, 1), (1222, 1), (1232, 1)]
[]
[]
[]
[]
[]
[]
[(96, 1), (334, 2)]
[]
[]
[]
[(216, 1)]
[]
[]
[(1232, 1)]
[]
[]
[]
[(1618, 1)]
[]
[]
[(220, 1), (334, 1), (1672, 1)]
[]
[]
[]
[]
[]
[]
[(334, 2)]
[]
[]
[]
[(415, 1)]
[]
[]
[(334, 2)]
[]
[]
[]
[(1618, 1)]
[]
[]
[(334, 2)]
[]
[]
[]
[]
[]
[]
[(334, 2)]
[]
[]
[]
[]
[]
[]
[(334, 2)]
[]
[]
[]
[]
[]
[]
[(28, 1), (334, 1), (1232, 1), (1244, 1)]
[]
[]
[]
[]
[]
[]
[(28, 1)]
[]
[]
[]
[]
[]
[]
[(188, 1), (334, 1), (339, 1), (849, 1), (984, 1), (1232, 2)]
[]
[]
[]
[(1232, 1)]
[]
[]
[(334, 2)]
[]
[]
[]
[]
[]
[]
[(334, 1), (1071, 1)]
[]
[]
[]
[]
[]
[]
[(260, 1), (334, 1), (1524, 1), (15

[]
[(344, 1), (397, 1), (516, 1), (968, 1), (1400, 1)]
[]
[]
[]
[(1113, 1)]
[]
[]
[(334, 1), (1121, 1)]
[]
[]
[]
[]
[]
[]
[(96, 2), (334, 1), (1121, 1)]
[]
[]
[]
[(93, 1), (212, 1), (1303, 1)]
[]
[]
[(334, 1)]
[]
[]
[]
[(93, 1)]
[]
[]
[(93, 2), (96, 1), (309, 1), (334, 1), (1121, 1), (1702, 1)]
[]
[]
[]
[(93, 1), (212, 1)]
[]
[]
[(212, 1), (334, 1), (1087, 1), (1121, 1)]
[]
[]
[]
[(746, 1), (857, 1)]
[]
[]
[(334, 1), (1087, 1), (1121, 1)]
[]
[]
[]
[(358, 1)]
[]
[]


In [8]:
from gensim.models import LdaModel

lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10)
print(lda_model)

LdaModel(num_terms=1863, num_topics=10, decay=0.5, chunksize=2000)


In [9]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [10]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(10,)
1.0
(10, 1863)
1.0


In [11]:
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
#pyLDAvis.display(prepared_data)
pyLDAvis.show(prepared_data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8898/    [Ctrl-C to exit]


127.0.0.1 - - [21/May/2019 19:39:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2019 19:39:30] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2019 19:39:30] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2019 19:39:30] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2019 19:39:30] code 404, message Not Found
127.0.0.1 - - [21/May/2019 19:39:30] "GET /favicon.ico HTTP/1.1" 404 -
